In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords

In [2]:
train = pd.read_csv(r"C:\Users\USER\OneDrive\Desktop\Kaggle\pop\labeledTrainData.tsv\labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)

In [10]:
test = pd.read_csv(r"C:\Users\USER\OneDrive\Desktop\Kaggle\pop\testData.tsv\testData.tsv", header=0, delimiter="\t", quoting=3 )

In [4]:
def review_to_words( raw_review ):
    # 1.HTML 지우기
    review_text = BeautifulSoup(raw_review).get_text() 
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    # 3. 소문자로, 단어분할
    words = letters_only.lower().split()                             
    # 4. convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words ))

In [5]:
num_reviews = train["review"].size
clean_train_reviews = []

# 모든 리뷰에 대해 review_to_words 적용, 리스트로 합치기
for i in range( 0, num_reviews ):
    clean_train_reviews.append( review_to_words( train["review"][i] ) )

In [6]:
from sklearn.model_selection import train_test_split

class_df = train['sentiment']
feature_df = clean_train_reviews

x_train, x_test, y_train, y_test = train_test_split(feature_df, class_df, test_size=0.3, random_state=11)

len(x_test)

7500

In [7]:
import sklearn

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

pipeline = Pipeline([('tfidf_vect', TfidfVectorizer(ngram_range=(1,2))), ('lr_clf', LogisticRegression(solver='liblinear', C=10))])

pipeline.fit(x_train, y_train)
pred = pipeline.predict(x_test)
pred_probs = pipeline.predict_proba(x_test)[:,1]

print(f'예측정확도는 {accuracy_score(y_test, pred):.4f}, ROC-AUC는 {roc_auc_score(y_test, pred_probs):.4f}')

예측정확도는 0.8977, ROC-AUC는 0.9622


In [12]:
# Create an empty list and append the clean reviews one by one
num_reviews = len(test["review"])
clean_test_reviews = [] 

print ("Cleaning and parsing the test set movie reviews...\n")
for i in range(0,num_reviews):
    if( (i+1) % 1000 == 0 ):
        print ("Review %d of %d\n" % (i+1, num_reviews))
    clean_review = review_to_words( test["review"][i] )
    clean_test_reviews.append( clean_review )

# Get a bag of words for the test set, and convert to a numpy array
# fit_transform()이 아니라 transform만 해야된다. 이미 train에서 fit했으니까 그 fit한걸로 test를 변환해야하는듯.

result = pipeline.predict_proba(clean_test_reviews)[:,1]

# Copy the results to a pandas dataframe with an "id" column and
# a "sentiment" column
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )

# Use pandas to write the comma-separated output file
output.to_csv( "Bag_of_Words_model_logistic.csv", index=False, quoting=3 )

Cleaning and parsing the test set movie reviews...

Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000

Review 6000 of 25000

Review 7000 of 25000

Review 8000 of 25000

Review 9000 of 25000

Review 10000 of 25000

Review 11000 of 25000

Review 12000 of 25000

Review 13000 of 25000

Review 14000 of 25000

Review 15000 of 25000

Review 16000 of 25000

Review 17000 of 25000

Review 18000 of 25000

Review 19000 of 25000

Review 20000 of 25000

Review 21000 of 25000

Review 22000 of 25000

Review 23000 of 25000

Review 24000 of 25000

Review 25000 of 25000



In [30]:
type(x_test)

list